In [1]:
from hand_detection import preprocess
from rephrase.paraphrase import Paraphrase
import tensorflow as tf
from hand_detection.preprocess import *
import cv2
import numpy
import math


model = tf.keras.models.load_model('./sign-classification/SavedModel/')
HANDMARKER = HandMarker()

2023-07-10 23:00:09.520342: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 23:00:09.583672: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 23:00:09.584360: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 23:00:10.525154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/cmaspi/.local/share/virtualenvs/hand-detection-advanced-8KyuWID6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as n

In [2]:
raised_hand = []

In [3]:
cap = cv2.VideoCapture('/home/cmaspi/Projects/hand-detection-advanced/test_vdeo.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
fps = math.ceil(fps)
frame_count = 0
print(f'FPS: {fps}')
if not cap.isOpened():
    print('Error opening the video stream')

print('Starting Processing the video!')
while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    try:
        _ = HANDMARKER.give_box(img)
        if (raised_hand and not raised_hand[-1][0]) or not raised_hand:
            if raised_hand and frame_count/fps - raised_hand[-1][1] < 1:
                raised_hand.pop(-1)
            else:
                timestamp = frame_count/fps
                raised_hand.append((True, timestamp))
            
    except:
        if (raised_hand and raised_hand[-1][0]) or not raised_hand:
            if not (raised_hand and frame_count/fps - raised_hand[-1][1] < 1):
                timestamp = frame_count/fps
                raised_hand.append((False, timestamp))

print(raised_hand)        

FPS: 30
Starting Processing the video!
Can't receive frame (stream end?). Exiting ...
[(False, 0.03333333333333333), (True, 2.1333333333333333), (False, 6.566666666666666), (True, 10.1), (False, 13.933333333333334), (True, 16.233333333333334), (False, 21.766666666666666), (True, 23.966666666666665)]


In [4]:
information = {}
num_hand_raised = sum([1 if i[0] else 0 for i in raised_hand])

information['How many times was the hand raised?'] = num_hand_raised
information['Was the hand raised at that moment?'] = None
information['n seconds into the video was the hand raised?'] = None


In [5]:
answers = {}
answers['How many times was the hand raised?'] = f'In the given video, we observe that the hand was raised {num_hand_raised} times'
answers['Was the hand raised at that moment?'] = None

In [6]:
questions = ['How many time the hand rose?',
             'at 5 seconds was the hand raised']


In [7]:
from Sentence_similarity.similarity import sim

for q in questions:
    probs = np.zeros(len(information.keys()))
    for i, k in enumerate(information.keys()):
        probs[i] = sim(q, k)
    idx = np.argmax(probs)
    if idx in [1, 2]:
        moment = int(' '.join([i for i in q if i.isdigit()]))
        hr = False
        for hand_raised, timestamp in raised_hand:
            if timestamp <= moment:
                hr = hand_raised
            else:
                break
        if hr:
            ans = f'Yes, about {moment} seconds into the provided video, we can see that the hand was raised.'
        else:
            ans = f'No, about {moment} seconds into the provided video, we can see that the hand was not raised.'
            
    else:
        ans = answers[list(information.keys())[idx]]
    possibilities = Paraphrase(ans)
    len_pos = len(possibilities)
    idx = np.random.choice(len_pos)
    ans = possibilities[idx][0]
    print(ans)

in the video we observe that the hand was raised 4 times
yes about 5 seconds into the video we can see the hand being raised
